In [1]:
import requests
import pandas as pd
import datetime as datetime
from pyspark.sql import SparkSession

In [146]:
'''
dt takes a format of either YYYY-MM-DD or YYYY-MM-DDTHH:mm:ss
and returns response as JSON if successful, else print error message and return None
'''
def extract_data_from_weather_api_as_json(url, params):
    weather_response = requests.get(url, params = params)
    response_status = weather_response.status_code
    try:
        weather_json = weather_response.json()
    except:
        print(f'Error: Response cannot be converted to JSON object.')
        return
    if response_status == 200:
        return weather_json
    elif response_status == 400 or response_status == 404:
        print(f'Error {response_status}: {weather_json['errorMsg']}')
    else:
        print(f'Error {response_status}: check your inputs')
    

In [ ]:
# Converts weather JSON for
def convert_weather_api_data_to_df(url, params):
    weather_json = extract_data_from_weather_api_as_json(url, params)
    if not weather_json:
        print('API call did not provide valid response to be converted to JSON')
    data = weather_json['data']
    pagination_token = weather_json['data'].get('PaginationToken')
    if pagination_token:
        while pagination_token:
            lst_of_readings = data['readings'] # up to 100 readings at once. need pagination token to go next
            

In [154]:
# 1 time extract data from 2020 to 2024
wind_speed_json_by_min = extract_data_from_weather_api_as_json('https://api-open.data.gov.sg/v2/real-time/api/wind-speed', {'date': '2025-01-01T20:12:00'})
station_info = pd.DataFrame(wind_speed_json_by_min['data']['stations']) # timestamp of reading in YYYY-MM-DDTHH:MM:SS±HH:MM format
station_info[['latitude', 'longitude']] = pd.DataFrame(station_info['location'].tolist()) # unpack geospatial coordinates
station_info.drop('location', inplace = True, axis = 1) # remove location column

date_extracted = wind_speed_json_by_min['data']['readings'][0]['timestamp']
station_id_and_reading = pd.DataFrame(wind_speed_json_by_min['data']['readings'][0]['data']) # stationID and reading
station_id_and_reading['timestamp'] = date_extracted

In [173]:
pd.concat([pd.DataFrame(val['data']).assign(timestamp = val['timestamp']) for val in wind_speed_json_by_min['data']['readings']])

,stationId,value,timestamp
0,S108,7.9,2025-01-01T23:59:00+08:00
1,S109,3.1,2025-01-01T23:59:00+08:00
2,S106,2.7,2025-01-01T23:59:00+08:00
3,S117,1.8,2025-01-01T23:59:00+08:00
4,S107,5.6,2025-01-01T23:59:00+08:00
...,...,...,...
9,S50,3.1,2025-01-01T22:20:00+08:00
10,S43,4.8,2025-01-01T22:20:00+08:00
11,S06,1.8,2025-01-01T22:20:00+08:00
12,S111,4.7,2025-01-01T22:20:00+08:00


In [159]:
# 1 time extract data from 2020 to 2024
wind_speed_json_by_min = extract_data_from_weather_api_as_json('https://api-open.data.gov.sg/v2/real-time/api/wind-speed', {'date': '2025-01-01'})
station_info = pd.DataFrame(wind_speed_json_by_min['data']['stations']) # timestamp of reading in YYYY-MM-DDTHH:MM:SS±HH:MM format
station_info[['latitude', 'longitude']] = pd.DataFrame(station_info['location'].tolist()) # unpack geospatial coordinates
station_info.drop('location', inplace = True, axis = 1) # remove location column

date_extracted = wind_speed_json_by_min['data']['readings'][0]['timestamp']
station_id_and_reading = pd.DataFrame(wind_speed_json_by_min['data']['readings'][0]['data']) # stationID and reading
station_id_and_reading['timestamp'] = date_extracted


In [ ]:
wind_speed_json['data']['paginationToken']#['data']['readings'][-1]
test = requests.get('https://api-open.data.gov.sg/v2/real-time/api/wind-speed', params = {'date': '2025-01-01', 'paginationToken': 'b2Zmc2V0PTEwMA=='}).json()
#extract_data_from_weather_api_as_json('https://api-open.data.gov.sg/v2/real-time/api/wind-speed', params = '2025-01-01', )

In [82]:
test['data']['paginationToken']

'b2Zmc2V0PTIwMA=='